In [ ]:
# !pip install kr8s

In [ ]:
!pip list | grep kr8s

In [1]:
import kr8s

# for node in kr8s.get("nodes"):
#     print(node.name)

In [2]:
# !mkdir serviceaccount
# kubectl get secret worker-manager-token -o jsonpath="{.data.token}" | base64 --decode > serviceaccount/token
# kubectl get secret worker-manager-token -o jsonpath='{.data.ca\.crt}' | base64 --decode > serviceaccount/ca.crt

In [6]:
import kr8s

client = kr8s.api(
    serviceaccount="./serviceaccount"
)

In [7]:
client.get("nodes")

[<Node k3d-syft-dev-server-0>]

In [8]:
from kr8s.objects import Pod

backend_pod = Pod.get("backend-0")

In [9]:
backend_pod

<Pod backend-0>

In [12]:
backend_pod.name

'backend-0'

In [13]:
backend_pod.namespace

'syft'

In [14]:
s = backend_pod.spec

In [15]:
m = backend_pod.metadata

In [17]:
m.to_dict()["labels"]

{'app.kubernetes.io/component': 'backend',
 'app.kubernetes.io/managed-by': 'Helm',
 'app.kubernetes.io/name': 'devspace-app',
 'controller-revision-hash': 'backend-b9d4458f8',
 'statefulset.kubernetes.io/pod-name': 'backend-0'}

In [20]:
# backend_pod.metadata

In [21]:
image = s.containers[0].image
image

'k3d-registry.localhost:12345/openmined/grid-backend:dev-1703048319'

In [30]:
env = s.containers[0].env
env

BoxList([{'name': 'PODNAME', 'valueFrom': {'fieldRef': {'apiVersion': 'v1', 'fieldPath': 'metadata.name'}}}, {'name': 'MONGO_PORT', 'value': '27017'}, {'name': 'MONGO_HOST', 'value': 'mongo'}, {'name': 'MONGO_USERNAME', 'value': 'root'}, {'name': 'MONGO_PASSWORD', 'valueFrom': {'secretKeyRef': {'name': 'mongosecret', 'key': 'mongo-password-secret'}}}, {'name': 'SERVICE_NAME', 'value': 'backend'}, {'name': 'RELEASE', 'value': 'production'}, {'name': 'VERSION', 'value': 'dev-1703048319'}, {'name': 'VERSION_HASH', 'value': 'abc'}, {'name': 'NODE_TYPE', 'value': 'domain'}, {'name': 'NODE_NAME', 'value': 'mynode'}, {'name': 'NODE_SIDE_TYPE', 'value': 'high'}, {'name': 'STACK_API_KEY', 'valueFrom': {'secretKeyRef': {'name': 'syftsecret', 'key': 'stackApiKey'}}}, {'name': 'PORT', 'value': '80'}, {'name': 'IGNORE_TLS_ERRORS', 'value': 'False'}, {'name': 'HTTP_PORT', 'value': '80'}, {'name': 'HTTPS_PORT', 'value': '443'}, {'name': 'QUEUE_PORT', 'value': '5556'}, {'name': 'CREATE_PRODUCER', 'val

In [27]:
import copy

In [31]:
new_env = copy.deepcopy(env)

In [32]:
for item in new_env:
    if item["name"] == "N_CONSUMERS":
        item["value"] = "1"
    if item["name"] == "CREATE_PRODUCER":
        item["value"] = "False"

In [33]:
new_env

BoxList([{'name': 'PODNAME', 'valueFrom': {'fieldRef': {'apiVersion': 'v1', 'fieldPath': 'metadata.name'}}}, {'name': 'MONGO_PORT', 'value': '27017'}, {'name': 'MONGO_HOST', 'value': 'mongo'}, {'name': 'MONGO_USERNAME', 'value': 'root'}, {'name': 'MONGO_PASSWORD', 'valueFrom': {'secretKeyRef': {'name': 'mongosecret', 'key': 'mongo-password-secret'}}}, {'name': 'SERVICE_NAME', 'value': 'backend'}, {'name': 'RELEASE', 'value': 'production'}, {'name': 'VERSION', 'value': 'dev-1703048319'}, {'name': 'VERSION_HASH', 'value': 'abc'}, {'name': 'NODE_TYPE', 'value': 'domain'}, {'name': 'NODE_NAME', 'value': 'mynode'}, {'name': 'NODE_SIDE_TYPE', 'value': 'high'}, {'name': 'STACK_API_KEY', 'valueFrom': {'secretKeyRef': {'name': 'syftsecret', 'key': 'stackApiKey'}}}, {'name': 'PORT', 'value': '80'}, {'name': 'IGNORE_TLS_ERRORS', 'value': 'False'}, {'name': 'HTTP_PORT', 'value': '80'}, {'name': 'HTTPS_PORT', 'value': '443'}, {'name': 'QUEUE_PORT', 'value': '5556'}, {'name': 'CREATE_PRODUCER', 'val

In [ ]:
# pod = Pod.gen(name="worker", image=image, env=new_env)

In [34]:
volume_mount = backend_pod.spec.containers[0]["volumeMounts"][0].to_dict()
volume_mount

{'name': 'credentials-data',
 'mountPath': '/root/data/creds/',
 'subPath': 'credentials-data'}

In [35]:
volume = backend_pod.spec.volumes[0].to_dict()
volume

{'name': 'credentials-data',
 'persistentVolumeClaim': {'claimName': 'credentials-data-backend-0'}}

In [65]:
name = "worker"
pod_dict = {
    "apiVersion":"v1",
    "kind":"Pod",
    "metadata":{
        "name":name,
    },
    "spec":{
        "volumes": [
            volume
        ],
        "containers":[
            {
                "name":name,
                "image":image,
                "env":new_env.to_list(),
                "volumeMounts": [
                    volume_mount
                ]
            }
        ],
        "restartPolicy":"Always",
    }
}

In [66]:
# worker_name = "worker"
# pod_dict = {
#     "apiVersion":"v1",
#     "kind":"Pod",
#     "metadata":{
#         "name":worker_name,
#     },
#     "spec":{
#         "containers":[
#             {
#                 "name":worker_name,
#                 "image":image,
#                 "env":new_env.to_list(),
#             }
#         ],
#         "restartPolicy":"Always",
#     }
# }

In [69]:
pod = Pod(pod_dict)

In [70]:
# pod = Pod.gen(name=worker_name, image=image, env=new_env)

In [71]:
json.dumps(pod.raw)

'{"apiVersion": "v1", "kind": "Pod", "metadata": {"name": "worker"}, "spec": {"volumes": [{"name": "credentials-data", "persistentVolumeClaim": {"claimName": "credentials-data-backend-0"}}], "containers": [{"name": "worker", "image": "k3d-registry.localhost:12345/openmined/grid-backend:dev-1703048319", "env": [{"name": "PODNAME", "valueFrom": {"fieldRef": {"apiVersion": "v1", "fieldPath": "metadata.name"}}}, {"name": "MONGO_PORT", "value": "27017"}, {"name": "MONGO_HOST", "value": "mongo"}, {"name": "MONGO_USERNAME", "value": "root"}, {"name": "MONGO_PASSWORD", "valueFrom": {"secretKeyRef": {"name": "mongosecret", "key": "mongo-password-secret"}}}, {"name": "SERVICE_NAME", "value": "backend"}, {"name": "RELEASE", "value": "production"}, {"name": "VERSION", "value": "dev-1703048319"}, {"name": "VERSION_HASH", "value": "abc"}, {"name": "NODE_TYPE", "value": "domain"}, {"name": "NODE_NAME", "value": "mynode"}, {"name": "NODE_SIDE_TYPE", "value": "high"}, {"name": "STACK_API_KEY", "valueFr

In [72]:
pod.create()

In [ ]:
type(pod)

In [ ]:
dir(pod)

In [ ]:
search_uid = pod.metadata.uid

In [ ]:
pods = kr8s.get("pods", namespace=kr8s.ALL)

In [ ]:
pods

In [ ]:
found_pod = None
for pod in pods:
    if pod.metadata.uid == search_uid:
        print("found", pod)
        found_pod = pod

In [ ]:
found_pod.delete()

In [ ]:
dir(found_pod)

In [ ]:
# Filter pods by UID
matching_pod = next((pod for pod in all_pods if pod['metadata']['uid'] == search_uid), None)

if matching_pod:
    print("Found Pod:", matching_pod['metadata']['name'], "in Namespace:", matching_pod['metadata']['namespace'])
else:
    print("No pod found with UID", search_uid)